# INSTALL: packages

In [1]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\info\appdata\local\programs\python\python310\lib\site-packages (1.4.2)



You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import os
import pickle
import WalletClustering_neo4jConnect

In [4]:
conn = WalletClustering_neo4jConnect.conn

# DEFINE: get interesting addresses excluding blacklisted addresses

## create output directory

In [5]:
try:
    os.mkdir('output')
except:
    pass

## query terror addresses

In [6]:
#The below code is returning a list of all the addresses that are marked as terror addresses.
#Query takes around 10 min
def returnTerrorAddresses():

    query_string = '''
    MATCH (a:Address {isTerror: True})
    Return a.address
    '''

    response = conn.query(query_string, db='neo4j')
    terrorAddresses = [r[0] for r in response]
    return terrorAddresses

In [7]:
# comment out entire block if old terrorAddressList saved as pickle should be used
def createTerrorAddressList():
    terrorAddressList = returnTerrorAddresses()
  
    # save terrorAddressList to file
    with open('output\\terrorAddressList.pickle', 'wb') as export:
        pickle.dump(terrorAddressList, export)

## create blacklist from csv

In [8]:
# allows blacklist of addresses in json format
def createBlacklistCSV():
    blacklistAddresses = pd.DataFrame(columns =['address', 'association'])

    dirname = os.path.dirname(os.path.realpath('__file__'))

    if os.path.isdir('..\\EntityAddressBitcoin\\'):        
        df = pd.read_csv(os.path.join(os.path.join(dirname, '..\\EntityAddressBitcoin\\'), 'Exchanges_full_detailed.csv'), usecols=['association', 'hashAdd'])[['hashAdd', 'association']]
        df.columns=['address', 'association']
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)

    #export blacklistAddresses to file
    with open('output\\blacklistAddresses.pickle', 'wb') as export:
        pickle.dump(blacklistAddresses, export)

## create blacklist from json

In [9]:
# allows blacklist of addresses in json format
def createBlacklist():
  
  # first create CSV blacklist, then append JSON
  createBlacklistCSV()
  
  if os.path.exists('output\\blacklistAddresses.pickle'):
    blacklistAddresses = pickle.load(open('output\\blacklistAddresses.pickle', 'rb'))
  else:
    blacklistAddresses = pd.DataFrame(columns =['address', 'association'])
  
  dirname = os.path.dirname(os.path.realpath('__file__'))
  
  # iterate over collection of exchange addresses in json format to create blacklist
  if os.path.isdir('..\\WalletExplorerScraper\\Output\\exchange\\'):
    outputScrapeExchangeFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\exchange\\')
    for subfolder in os.listdir(outputScrapeExchangeFolder):
        print(subfolder)
        df = pd.read_json(os.path.join(outputScrapeExchangeFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue
  
  if os.path.isdir('..\\WalletExplorerScraper\\Output\\mixer\\'):
    outputScrapeMixerFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\mixer\\')
    for subfolder in os.listdir(outputScrapeMixerFolder):
        print(subfolder)
        df = pd.read_json(os.path.join(outputScrapeMixerFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue
  
  if os.path.isdir('..\\WalletExplorerScraper\\Output\\giant_wallet\\'):
    outputScrapeGiantWalletFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\giant_wallet\\')
    for subfolder in os.listdir(outputScrapeGiantWalletFolder):
        print(subfolder)
        df = pd.read_json(os.path.join(outputScrapeGiantWalletFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue

  if os.path.isdir('..\\WalletExplorerScraper\\Output\\other_service\\'):
    outputScrapeOtherServiceFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\other_service\\')
    for subfolder in os.listdir(outputScrapeOtherServiceFolder):
        print(subfolder)
        df = pd.read_json(os.path.join(outputScrapeOtherServiceFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue
      
  blacklistAddresses['association'] = blacklistAddresses['association'].str.lower()
  blacklistAddresses = blacklistAddresses.drop_duplicates('address', keep='last')

  #export blacklistAddresses to file
  with open('output\\blacklistAddresses.pickle', 'wb') as export:
    pickle.dump(blacklistAddresses, export)

## create list of blacklisted associations

In [10]:
def createAssocBlacklist():
    assocBlacklistDf = pickle.load(open('output\\blacklistAddresses.pickle', 'rb'))
    assocBlacklistDf = assocBlacklistDf.drop_duplicates('association', keep='first')
    assocBlacklist = assocBlacklistDf['association'].tolist()
    
    otherServicesListDf = pickle.load(open('output/blacklistAddressesOSMIH.pickle', 'rb')) # list of other services addresses that have been clustered with MIH
    otherServicesListDf = otherServicesListDf.drop_duplicates('otherService', keep='last')
    otherServicesList = otherServicesListDf['otherService'].tolist()
    assocBlacklist.extend(otherServicesList)

    with open('output\\assocBlacklist.pickle', 'wb') as export:
        pickle.dump(list(set(assocBlacklist)), export)


## remove blacklisted addresses from addresses of interest

In [11]:
# to be used for removal of exchange addresses
def addressCleanUp(addressList, blacklist):
    interestingAddresses = []
    for address in addressList:
        if not blacklist['address'].str.contains(address).any():
            interestingAddresses.append(address)
    return interestingAddresses

In [12]:
def createAddressesToClusterList():
    addressesToCluster = addressCleanUp(terrorAddressList, blacklistAddresses)

    # save addressesToCluster to file
    with open('output\\addressesToCluster.pickle', 'wb') as export:
        pickle.dump(addressesToCluster, export)

# EXECUTE: get interesting addresses excluding blacklisted addresses

In [13]:
# use existing terrorAddressList if exists
if not os.path.exists('output\\terrorAddressList.pickle'):
    createTerrorAddressList()

terrorAddressList = pickle.load(open('output\\terrorAddressList.pickle', 'rb'))
print(terrorAddressList)

['12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8', '1348ThkNoDupq1bws95diMiL8haGs61K7M', '13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc', '13Pcmh4dKJE8Aqrhq4ZZwmM1sbKFcMQEEV', '15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf', '15soXrE3NJBMkkQhrccXonTT9bpjpPvE67', '164fawNZVwsR5SamAJypvCMtkMx4Xv1B3f', '179bzhS4FY7qLDza9YjuorhWyXVVYZu2YH', '17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD', '17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr', '19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW', '19XVEDZCGVMA9WCF1qUayxtnjUnyD7zDDQ', '1A7pDH1EdrkH9YZtsPnc8uzirBFnAN9Eay', '1BPf9qr7M5xUgNHUYtrQtEKvUKcyERzXao', '1C6hetVWVXZnS6P2BYBNu5Y1ZJ57JyXGac', '1DrhHEkv42JVwiDQNi28JFdSuiSGgPNXwP', '1EDcKCRypUTFoTZbxDWF9MBAT4W7XUGB32', '1EfmRn6Bp3cjrTBubaH8MzRRc2ikSjNGXw', '1EnX6BuJiGWydqXJT9BN5dSvfLg3QW4Mdz', '1EVTZmTMqZPMzGxsug9TXBtvPJZH8dXSCK', '1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR', '1GALPyvUDDXqA6H2eHQ9Y1yidfQ6T1Drvn', '1GC2SjzCyCwxo1uxTi28oqn9L3mJj7bLPs', '1Gg25VzQkqCizXHNSNet4RoysLEe19su4s', '1JpSBaUwrZaEgmsYka7mzm9t3Z4syyaw7A', '1LhRW1msre1cFgT7fBY2BRrZ4ANMPwVj9u', '1Lm9BCDUKo

In [14]:
# use existing blacklist if exists
if not os.path.exists('output\\blacklistAddresses.pickle'):
    createBlacklist()

blacklistAddresses = pickle.load(open('output\\blacklistAddresses.pickle', 'rb'))
print(blacklistAddresses)

C:\Users\info\AppData\Local\Temp\ipykernel_12136\2051127507.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)


0_huobi.com


C:\Users\info\AppData\Local\Temp\ipykernel_12136\4096622187.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
C:\Users\info\AppData\Local\Temp\ipykernel_12136\4096622187.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)


100_banx.io
101_banx.io


C:\Users\info\AppData\Local\Temp\ipykernel_12136\4096622187.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)


...

0_eligius.st


C:\Users\info\AppData\Local\Temp\ipykernel_12136\4096622187.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)


1_epay.info


C:\Users\info\AppData\Local\Temp\ipykernel_12136\4096622187.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)


                                     address  association
1         1CrhRvGShwmQkrGcqFzDDwDDBiEhKpHAqF   anxpro.com
2         1AbH5pFqocUYVhRqPAtJruVr5DMAxEANmn   anxpro.com
4         1FM8vDXinj2N5LoTXX5J4MSGTCn2pPkqRm   anxpro.com
7         18g2RSp4MxzkyNkigpYDejP2jMgkJ2Po9A   anxpro.com
18        195rycdSmRf1tQDztryRkZWKaFgFwM1PPG  bittrex.com
...                                      ...          ...
21616496   1zFCx8MQEx34Jaw1b25vZUsf1ojVVtwaF    epay.info
21616497  1HkmSR4vdZuYPEdBWHDpHeQe51aXenvHnK    epay.info
21616498  1CyHtTtHNBGabRbDrdf5t8Nij4TNJN7hi4    epay.info
21616499  1BqciLwW1rm7GVWPyqgkhqtqYdhy1sSgtK    epay.info
21616500  12D5aJQQ7JsRdh42wdMwBFLYVLSFmh2RY2    epay.info

[13993053 rows x 2 columns]


In [15]:
if not os.path.exists('output\\assocBlacklist.pickle'):
    createAssocBlacklist()

assocBlacklist = pickle.load(open('output\\assocBlacklist.pickle', 'rb'))
print(assocBlacklist)

['coinurl.com', 'bitcoinp2p.com.br', 'unknown_large_wallet_02', 'coinjar.com', 'cryptopay.me', 'orderbook.net', 'anonibet.com', 'coinmkt.com', 'haobtc.com', 'bitlaunder.com', 'bleutrade.com', 'paxful.com', 'btc38.com', 'antpool.com', 'cannabisroadmarket', 'betcoin.tm', 'bitnz.com', 'coinhako.com', 'coinbox.me', 'yobit.net', 'blackbankmarket', 'btcguild.com', 'btcc.com', 'btcmarkets.net', 'updown.bt', 'masterxchange.com', 'okcoin.com', 'bit-x.com', 'coinomat.com', 'bitoex.com', 'coingaming.io', 'coinkite.com', 'bitcoinica.com', 'bitoomba.com', 'hitbtc.com', 'securevpn.to', 'btcoracle.com', 'bitmillions.com', 'chainroll.com', 'coinroyale.com', 'satoshimines.com', 'bitcoinvietnam.com.vn', 'betchain.com', 'virwox.com', 'coinvault', 'bitvc.com', 'btcexchange.ro', 'btcst.com', 'everydice.com', 'crimenetwork.cc', 'oklink.com', 'campbx.com', 'diceoin.io', 'silkroad2market', 'satoshicircle.com', 'bitcoin-roulette.com', 'coinmate.io', 'sealswithclubs.eu', 'just-dice.com', 'playt.in', 'crimenetwo

In [16]:
# use existing addressesToCluster if exists
if not os.path.exists('output\\addressesToCluster.pickle'):
    createAddressesToClusterList()

addressesToCluster = pickle.load(open('output\\addressesToCluster.pickle', 'rb'))
print(addressesToCluster)

['12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8', '1348ThkNoDupq1bws95diMiL8haGs61K7M', '13Pcmh4dKJE8Aqrhq4ZZwmM1sbKFcMQEEV', '15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf', '164fawNZVwsR5SamAJypvCMtkMx4Xv1B3f', '179bzhS4FY7qLDza9YjuorhWyXVVYZu2YH', '17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD', '17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr', '19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW', '19XVEDZCGVMA9WCF1qUayxtnjUnyD7zDDQ', '1BPf9qr7M5xUgNHUYtrQtEKvUKcyERzXao', '1C6hetVWVXZnS6P2BYBNu5Y1ZJ57JyXGac', '1DrhHEkv42JVwiDQNi28JFdSuiSGgPNXwP', '1EDcKCRypUTFoTZbxDWF9MBAT4W7XUGB32', '1EfmRn6Bp3cjrTBubaH8MzRRc2ikSjNGXw', '1EnX6BuJiGWydqXJT9BN5dSvfLg3QW4Mdz', '1EVTZmTMqZPMzGxsug9TXBtvPJZH8dXSCK', '1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR', '1GALPyvUDDXqA6H2eHQ9Y1yidfQ6T1Drvn', '1GC2SjzCyCwxo1uxTi28oqn9L3mJj7bLPs', '1Gg25VzQkqCizXHNSNet4RoysLEe19su4s', '1JpSBaUwrZaEgmsYka7mzm9t3Z4syyaw7A', '1LhRW1msre1cFgT7fBY2BRrZ4ANMPwVj9u', '1Lm9BCDUKoBUk888DCXewM5p8bJyr83cEp', '1LPTaRfyoNwvwAtmYzcetZLjBfUxVkJrr4', '1MMaU5nTrFdPZotfwdbv1wWnFjLCTFbpPY', '1uLdz4wXrc